In [ ]:
import torch as ch
from os import path

from prediction.short_term_outcome_prediction.timeseries_decomposition import prepare_subsequence_dataset

In [ ]:
data_splits_path = '/Users/jk1/Downloads/train_data_splits_early_neurological_deterioration_ts0.8_rs42_ns5.pth'

In [ ]:
use_gpu = ch.cuda.is_available()

In [ ]:
splits = ch.load(path.join(data_splits_path))
all_datasets = [prepare_subsequence_dataset(x, use_gpu=use_gpu, use_time_to_event=True) for x in splits]

In [ ]:
for i in range(len(all_datasets[0][0])):
    print(all_datasets[0][0][i][1])

In [ ]:
train_dataset = all_datasets[0][0]

In [ ]:
from prediction.outcome_prediction.Transformer.architecture import OPSUMTransformer

input_dim = train_dataset[0][0].shape[-1]
ff_factor = 2
ff_dim = ff_factor * 1024

model = OPSUMTransformer(
    input_dim=input_dim,
    num_layers=1,
    model_dim=1024,
    dropout=0.1,
    ff_dim=ff_dim,
    num_heads=2,
    num_classes=1,
    max_dim=500,
    pos_encode_factor=1,
    enforce_non_negative=True
)

In [ ]:
from torch.utils.data import DataLoader
from prediction.short_term_outcome_prediction.timeseries_decomposition import BucketBatchSampler

train_bucket_sampler = BucketBatchSampler(train_dataset.idx_to_len_map, 2)
train_loader = DataLoader(train_dataset, batch_sampler=train_bucket_sampler,
                                  # shuffling is done in the bucket sampler
                                  shuffle=False, drop_last=False)

In [ ]:
sample = next(iter(train_loader))

In [ ]:
sample[0].shape

In [ ]:
model(sample[0])

In [ ]:
model(sample[0]).shape

In [ ]:
from prediction.outcome_prediction.Transformer.lightning_wrapper import LitModel

module = LitModel(model, lr=0.0001, wd=0.1, train_noise=0.1) 

In [ ]:
module.predict_step(sample, 0).shape